# 🚀 Train Model và Upload lên Hugging Face

Notebook này giúp bạn:
1. ✅ Kiểm tra dataset
2. 🔄 Generate assistant responses (nếu cần)
3. 🎯 Train model với LoRA
4. 📤 Upload model lên Hugging Face

**Lưu ý:** Dataset cần có format Qwen3 với `messages` chứa `system`, `user`, và `assistant` roles.


## 1. Cài đặt Dependencies


In [ ]:
# Cài đặt các thư viện cần thiết
%pip install -q transformers>=4.41.0,<5.0.0
%pip install -q accelerate>=0.20.0
%pip install -q bitsandbytes>=0.41.0
%pip install -q peft>=0.6.0
%pip install -q datasets>=2.14.0
%pip install -q torch>=2.0.0
%pip install -q tensorboard>=2.13.0
%pip install -q huggingface_hub>=0.20.0
%pip install -q tqdm

print("✅ Đã cài đặt tất cả dependencies!")


## 2. Import Libraries


In [ ]:
import os
import json
import torch
from pathlib import Path
from collections import Counter
from transformers import (
    Qwen3VLForConditionalGeneration,
    AutoProcessor,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from datasets import Dataset, load_dataset
from tqdm import tqdm
from huggingface_hub import login, HfApi, create_repo

print("✅ Đã import tất cả libraries!")


## 3. Cấu Hình

Thay đổi các tham số dưới đây theo nhu cầu của bạn:


In [ ]:
# ============================================================================
# CẤU HÌNH MODEL
# ============================================================================
MODEL_NAME = "hainguyen306201/bank-model-2b"  # Model base để fine-tune

# ============================================================================
# CẤU HÌNH DATASET
# ============================================================================
# Cách 1: Upload file lên Colab (chạy cell upload bên dưới)
DATASET_PATH = "/content/dataset.jsonl"

# Cách 2: Load từ GitHub hoặc URL
# DATASET_URL = "https://raw.githubusercontent.com/username/repo/main/dataset.jsonl"

# ============================================================================
# CẤU HÌNH TRAINING
# ============================================================================
OUTPUT_DIR = "./checkpoints"
EPOCHS = 3
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 1
LEARNING_RATE = 2e-4
MAX_LENGTH = 2048
WARMUP_STEPS = 100
SAVE_STEPS = 500
LOGGING_STEPS = 50

# ============================================================================
# CẤU HÌNH LoRA
# ============================================================================
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# ============================================================================
# CẤU HÌNH HUGGING FACE
# ============================================================================
# Đặt token của bạn (hoặc login bằng huggingface-cli)
HF_TOKEN = ""  # Để trống nếu đã login
HUB_MODEL_ID = ""  # Ví dụ: "username/bank-model-finetuned"
HUB_PRIVATE = False  # True nếu muốn private repo
PUSH_TO_HUB = False  # True để upload sau khi train xong

# ============================================================================
# CẤU HÌNH KHÁC
# ============================================================================
USE_QUANTIZATION = True  # Dùng 4-bit quantization để tiết kiệm bộ nhớ
USE_FLASH_ATTENTION = False  # Cần cài flash-attn riêng

print("✅ Đã cấu hình xong!")


## 4. Upload Dataset (Nếu cần)

Nếu dataset của bạn ở local, upload lên Colab:


In [ ]:
from google.colab import files

# Upload file dataset
uploaded = files.upload()

# Tìm file vừa upload
for filename in uploaded.keys():
    print(f"✅ Đã upload: {filename}")
    DATASET_PATH = f"/content/{filename}"
    break

print(f"\n📂 Dataset path: {DATASET_PATH}")


## 5. Kiểm Tra Dataset


In [ ]:
def check_dataset(file_path: str):
    """Kiểm tra dataset và hiển thị thống kê"""
    print(f"📂 Đang kiểm tra dataset: {file_path}\n")
    
    if not Path(file_path).exists():
        print(f"❌ File không tồn tại: {file_path}")
        return False
    
    # Đọc dataset
    samples = []
    issues = []
    
    with open(file_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            line = line.strip()
            if not line:
                continue
            
            try:
                data = json.loads(line)
                samples.append((line_num, data))
            except json.JSONDecodeError as e:
                issues.append(f"Dòng {line_num}: JSON không hợp lệ - {e}")
    
    if not samples:
        print("❌ Dataset trống!")
        return False
    
    print(f"✅ Đã đọc {len(samples)} samples\n")
    
    # Kiểm tra format
    role_counts = Counter()
    has_system = 0
    has_user = 0
    has_assistant = 0
    missing_assistant = []
    
    for line_num, data in samples:
        if "messages" not in data:
            issues.append(f"Dòng {line_num}: Thiếu field 'messages'")
            continue
        
        messages = data.get("messages", [])
        if not isinstance(messages, list) or len(messages) == 0:
            issues.append(f"Dòng {line_num}: Messages rỗng hoặc không phải list")
            continue
        
        # Đếm roles
        roles = [msg.get("role") for msg in messages if isinstance(msg, dict)]
        role_counts.update(roles)
        
        # Kiểm tra có system/user/assistant không
        if any(msg.get("role") == "system" for msg in messages):
            has_system += 1
        if any(msg.get("role") == "user" for msg in messages):
            has_user += 1
        if any(msg.get("role") == "assistant" for msg in messages):
            has_assistant += 1
        else:
            missing_assistant.append(line_num)
    
    # Hiển thị thống kê
    print("=" * 60)
    print("📊 THỐNG KÊ DATASET")
    print("=" * 60)
    print(f"Tổng số samples: {len(samples)}")
    print(f"\nPhân bố roles:")
    for role, count in role_counts.most_common():
        print(f"  - {role}: {count} messages")
    
    print(f"\nSamples có:")
    print(f"  - System message: {has_system}/{len(samples)} ({has_system/len(samples)*100:.1f}%)")
    print(f"  - User message: {has_user}/{len(samples)} ({has_user/len(samples)*100:.1f}%)")
    print(f"  - Assistant message: {has_assistant}/{len(samples)} ({has_assistant/len(samples)*100:.1f}%)")
    
    # Cảnh báo
    print("\n" + "=" * 60)
    print("⚠️  CẢNH BÁO")
    print("=" * 60)
    
    if missing_assistant:
        print(f"\n❌ QUAN TRỌNG: {len(missing_assistant)} samples THIẾU assistant responses!")
        print("   Dataset cần có cả user và assistant messages để training hiệu quả.")
        if len(missing_assistant) <= 10:
            print(f"   Các dòng thiếu: {missing_assistant[:10]}")
        else:
            print(f"   Các dòng thiếu: {missing_assistant[:10]} ... và {len(missing_assistant) - 10} dòng khác")
        print("\n   💡 Giải pháp: Chạy cell 'Generate Responses' bên dưới")
    
    if has_assistant == 0:
        print("\n❌ Dataset KHÔNG CÓ assistant responses nào!")
        print("   Cần generate responses trước khi training.")
        return False
    
    if has_assistant < len(samples) * 0.9:
        print(f"\n⚠️  Chỉ {has_assistant}/{len(samples)} samples có assistant responses")
        print("   Nên có ít nhất 90% samples có assistant responses")
    
    if issues:
        print(f"\n⚠️  Có {len(issues)} vấn đề trong dataset:")
        for issue in issues[:10]:
            print(f"   - {issue}")
        if len(issues) > 10:
            print(f"   ... và {len(issues) - 10} vấn đề khác")
    
    # Kết luận
    print("\n" + "=" * 60)
    print("✅ KẾT LUẬN")
    print("=" * 60)
    
    if has_assistant == len(samples) and has_user == len(samples):
        print("✅ Dataset hợp lệ và sẵn sàng cho training!")
        return True
    elif has_assistant > 0:
        print("⚠️  Dataset có thể training nhưng không tối ưu")
        print("   Nên bổ sung assistant responses cho tất cả samples")
        return True
    else:
        print("❌ Dataset không thể training (thiếu assistant responses)")
        return False

# Kiểm tra dataset
dataset_valid = check_dataset(DATASET_PATH)


## 6. Generate Assistant Responses (Nếu cần)

**Chỉ chạy cell này nếu dataset thiếu assistant responses!**


In [ ]:
# Chỉ chạy nếu dataset thiếu assistant responses
GENERATE_RESPONSES = False  # Đặt True nếu cần generate

if GENERATE_RESPONSES:
    print("📥 Đang load model để generate responses...")
    
    # Load model
    model_gen = Qwen3VLForConditionalGeneration.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        torch_dtype=torch.bfloat16,
        trust_remote_code=True
    )
    processor_gen = AutoProcessor.from_pretrained(MODEL_NAME, trust_remote_code=True)
    
    print("✅ Model đã được load!\n")
    
    # Đọc dataset
    print(f"📂 Đang đọc dataset: {DATASET_PATH}...")
    samples = []
    with open(DATASET_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                samples.append(json.loads(line))
    
    print(f"✅ Đã đọc {len(samples)} samples\n")
    
    # Process từng sample
    print("🔄 Đang generate responses...")
    processed_samples = []
    
    def generate_response(messages, max_new_tokens=512):
        """Generate response từ model"""
        inputs = processor_gen.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_dict=True,
            return_tensors="pt"
        )
        inputs = {k: v.to(model_gen.device) for k, v in inputs.items()}
        
        with torch.no_grad():
            generated_ids = model_gen.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                pad_token_id=processor_gen.tokenizer.eos_token_id
            )
        
        input_length = inputs["input_ids"].shape[1]
        generated_ids_trimmed = generated_ids[:, input_length:]
        output_text = processor_gen.batch_decode(
            generated_ids_trimmed,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
        
        return output_text[0].strip()
    
    for i, sample in enumerate(tqdm(samples, desc="Generating")):
        if "messages" not in sample:
            continue
        
        messages = sample["messages"]
        
        # Kiểm tra xem đã có assistant response chưa
        has_assistant = any(msg.get("role") == "assistant" for msg in messages)
        if has_assistant:
            processed_samples.append(sample)
            continue
        
        # Generate response
        try:
            response = generate_response(messages, max_new_tokens=512)
            
            # Thêm assistant response
            new_messages = messages.copy()
            new_messages.append({
                "role": "assistant",
                "content": response
            })
            
            new_sample = {"messages": new_messages}
            if "metadata" in sample:
                new_sample["metadata"] = sample["metadata"]
            
            processed_samples.append(new_sample)
            
        except Exception as e:
            print(f"\n⚠️  Lỗi ở sample {i+1}: {e}")
            processed_samples.append(sample)
    
    # Lưu dataset mới
    output_path = DATASET_PATH.replace(".jsonl", "_with_responses.jsonl")
    print(f"\n💾 Đang lưu dataset mới: {output_path}...")
    with open(output_path, 'w', encoding='utf-8') as f:
        for sample in processed_samples:
            f.write(json.dumps(sample, ensure_ascii=False) + '\n')
    
    DATASET_PATH = output_path
    print(f"✅ Đã lưu {len(processed_samples)} samples vào {output_path}")
    print(f"✅ Dataset path mới: {DATASET_PATH}")
    
    # Kiểm tra lại
    check_dataset(DATASET_PATH)
else:
    print("⏭️  Bỏ qua generate responses (GENERATE_RESPONSES = False)")


In [ ]:
if PUSH_TO_HUB:
    if HF_TOKEN:
        login(token=HF_TOKEN)
        print("✅ Đã login bằng token")
    else:
        print("💡 Chạy lệnh sau để login:")
        print("   !huggingface-cli login")
        print("\nHoặc set HF_TOKEN trong cell cấu hình")
        # Uncomment dòng dưới nếu muốn login thủ công
        # login()
else:
    print("⏭️  Không cần login (PUSH_TO_HUB = False)")


## 8. Load Model và Processor


In [ ]:
print(f"📥 Đang load model: {MODEL_NAME}...")

# Cấu hình quantization
bnb_config = None
if USE_QUANTIZATION:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )
    print("✅ Sử dụng 4-bit quantization")

# Load model
model_kwargs = {
    "device_map": "auto",
    "torch_dtype": torch.bfloat16,
    "trust_remote_code": True,
}

if bnb_config:
    model_kwargs["quantization_config"] = bnb_config

model = Qwen3VLForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    **model_kwargs
)

# Load processor
processor = AutoProcessor.from_pretrained(MODEL_NAME, trust_remote_code=True)

print("✅ Model và processor đã được load!")


## 9. Setup LoRA


In [ ]:
print("🔧 Đang setup LoRA...")

# Chuẩn bị model cho training
model = prepare_model_for_kbit_training(model)

# Cấu hình LoRA
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM",
)

# Áp dụng LoRA
model = get_peft_model(model, lora_config)

# In thông tin
model.print_trainable_parameters()
print("✅ LoRA đã được setup!")


## 10. Load và Preprocess Dataset


In [ ]:
print(f"📂 Đang load dataset từ {DATASET_PATH}...")

# Load từ JSONL
data = []
with open(DATASET_PATH, 'r', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line:
            data.append(json.loads(line))

dataset = Dataset.from_list(data)
print(f"✅ Đã load {len(dataset)} samples")

# Preview một sample
print("\n📋 Preview sample đầu tiên:")
print(json.dumps(data[0], ensure_ascii=False, indent=2))


In [ ]:
def preprocess_function(examples):
    """Preprocess function cho dataset"""
    messages_list = examples["messages"]
    
    input_ids_list = []
    attention_mask_list = []
    labels_list = []
    
    for messages in messages_list:
        # Apply chat template
        inputs = processor.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=False,
            return_dict=True,
            return_tensors="pt"
        )
        
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs.get("attention_mask", None)
        if attention_mask is not None:
            attention_mask = attention_mask.squeeze(0)
        else:
            attention_mask = torch.ones_like(input_ids)
        
        # Labels: copy input_ids
        labels = input_ids.clone()
        
        input_ids_list.append(input_ids)
        attention_mask_list.append(attention_mask)
        labels_list.append(labels)
    
    # Pad sequences
    max_length = min(MAX_LENGTH, max(len(ids) for ids in input_ids_list))
    
    def pad_sequence(seq, max_len, pad_value=processor.tokenizer.pad_token_id):
        if len(seq) >= max_len:
            return seq[:max_len]
        padded = torch.cat([seq, torch.full((max_len - len(seq),), pad_value, dtype=seq.dtype)])
        return padded
    
    pad_token_id = processor.tokenizer.pad_token_id or processor.tokenizer.eos_token_id
    
    input_ids_padded = torch.stack([pad_sequence(ids, max_length, pad_token_id) for ids in input_ids_list])
    attention_mask_padded = torch.stack([pad_sequence(mask, max_length, 0) for mask in attention_mask_list])
    labels_padded = torch.stack([pad_sequence(labels, max_length, -100) for labels in labels_list])
    
    return {
        "input_ids": input_ids_padded,
        "attention_mask": attention_mask_padded,
        "labels": labels_padded,
    }

# Preprocess dataset
print("🔄 Đang preprocess dataset...")
processed_dataset = dataset.map(
    lambda x: preprocess_function({"messages": [x["messages"]]}),
    batched=False,
    remove_columns=dataset.column_names,
)
print("✅ Đã preprocess xong!")


## 11. Training Arguments


In [ ]:
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    logging_steps=LOGGING_STEPS,
    save_steps=SAVE_STEPS,
    save_total_limit=3,
    fp16=torch.cuda.is_available(),
    bf16=torch.cuda.is_available() and torch.cuda.get_device_capability()[0] >= 8,
    optim="adamw_torch",
    report_to="tensorboard",
    remove_unused_columns=False,
)

print("✅ Training arguments đã được cấu hình!")
print(f"\n📊 Thông tin training:")
print(f"   - Epochs: {EPOCHS}")
print(f"   - Batch size: {BATCH_SIZE}")
print(f"   - Gradient accumulation: {GRADIENT_ACCUMULATION_STEPS}")
print(f"   - Learning rate: {LEARNING_RATE}")
print(f"   - Output dir: {OUTPUT_DIR}")


## 12. Bắt Đầu Training 🚀


In [ ]:
# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=processor.tokenizer,
    mlm=False,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=processed_dataset,
    data_collator=data_collator,
)

# Training
print("\n" + "="*50)
print("🚀 BẮT ĐẦU TRAINING")
print("="*50)
print(f"Model: {MODEL_NAME}")
print(f"Dataset: {DATASET_PATH} ({len(dataset)} samples)")
print(f"Epochs: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Output: {OUTPUT_DIR}")
print("="*50 + "\n")

trainer.train()

# Save final model
print("\n💾 Đang lưu model cuối cùng...")
trainer.save_model()
processor.save_pretrained(OUTPUT_DIR)

print(f"\n✅ Training hoàn thành! Model đã được lưu tại: {OUTPUT_DIR}")


## 13. Upload lên Hugging Face 📤


In [ ]:
if PUSH_TO_HUB and HUB_MODEL_ID:
    print(f"📤 Đang upload model lên Hugging Face: {HUB_MODEL_ID}...")
    
    # Tạo repository nếu chưa tồn tại
    api = HfApi()
    try:
        create_repo(
            repo_id=HUB_MODEL_ID,
            repo_type="model",
            private=HUB_PRIVATE,
            exist_ok=True
        )
        print(f"✅ Repository {HUB_MODEL_ID} đã sẵn sàng")
    except Exception as e:
        print(f"⚠️  Repository có thể đã tồn tại hoặc có lỗi: {e}")
    
    # Upload model
    try:
        api.upload_folder(
            folder_path=OUTPUT_DIR,
            repo_id=HUB_MODEL_ID,
            repo_type="model",
            commit_message=f"Upload fine-tuned model after {EPOCHS} epochs training"
        )
        print(f"\n✅ Đã upload model thành công!")
        print(f"🔗 Link: https://huggingface.co/{HUB_MODEL_ID}")
    except Exception as e:
        print(f"❌ Lỗi khi upload model: {e}")
        print(f"💡 Bạn có thể upload thủ công bằng lệnh:")
        print(f"   !huggingface-cli upload {HUB_MODEL_ID} {OUTPUT_DIR} --repo-type model")
else:
    print("⏭️  Không upload (PUSH_TO_HUB = False hoặc HUB_MODEL_ID chưa được set)")
    if not HUB_MODEL_ID:
        print("💡 Đặt HUB_MODEL_ID trong cell cấu hình để upload")


## 14. Test Model (Tùy chọn)

Test model sau khi training:


In [ ]:
# Load model đã fine-tune để test
TEST_MODEL = False  # Đặt True để test

if TEST_MODEL:
    print("📥 Đang load model đã fine-tune...")
    
    test_model = Qwen3VLForConditionalGeneration.from_pretrained(
        OUTPUT_DIR,
        device_map="auto",
        torch_dtype=torch.bfloat16,
    )
    test_processor = AutoProcessor.from_pretrained(OUTPUT_DIR)
    
    # Test với một câu hỏi mẫu
    test_messages = [
        {
            "role": "system",
            "content": "Bạn là tư vấn viên ngân hàng châu Á, nói chuyện trang trọng, xưng hô anh/chị, giải thích rõ ràng về lãi suất và các gói vay."
        },
        {
            "role": "user",
            "content": "Lãi suất vay hiện tại là bao nhiêu?"
        }
    ]
    
    inputs = test_processor.apply_chat_template(
        test_messages,
        tokenize=True,
        add_generation_prompt=True,
        return_dict=True,
        return_tensors="pt"
    )
    inputs = {k: v.to(test_model.device) for k, v in inputs.items()}
    
    generated_ids = test_model.generate(**inputs, max_new_tokens=256)
    generated_ids_trimmed = [
        out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs["input_ids"], generated_ids)
    ]
    output_text = test_processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    
    print("\n" + "="*50)
    print("📝 TEST RESULT")
    print("="*50)
    print("Input:", test_messages[1]["content"])
    print("\nOutput:", output_text[0])
    print("="*50)
else:
    print("⏭️  Bỏ qua test (TEST_MODEL = False)")


## ✅ Hoàn Thành!

Model đã được train và upload (nếu có cấu hình). Bạn có thể:
- Sử dụng model từ `OUTPUT_DIR` để inference local
- Sử dụng model từ Hugging Face Hub nếu đã upload
- Tiếp tục fine-tune với dataset khác

**Lưu ý:** Nếu training trên Colab, nhớ download model về local trước khi session hết hạn!
